# BONUS: Timestamps, UTC i Auto Loader z Archiwizacją

Notatnik bonusowy dla Data Engineerów pracujących w środowisku Databricks.

Obejmuje dwa kluczowe tematy:
1. **Praca z datami i timestampami** w kontekście zespołów wielonarodowych — standardy UTC, konwersja stref czasowych, pułapki i najlepsze praktyki.
2. **Auto Loader z mechanizmem archiwizacji** — automatyczne przenoszenie przetworzonych plików do folderu archiwum (`cloudFiles.cleanSource = MOVE`).

| Poziom | Czas |
|--------|------|
| Intermediate / Advanced | ~45 min |

> **Dokumentacja:** [Auto Loader options — docs.databricks.com](https://docs.databricks.com/aws/en/ingestion/cloud-object-storage/auto-loader/options)

---

## Part 1: Timestamps i UTC

---

### Setup

Konfiguracja środowiska — importy, zmienne środowiskowe, ścieżki.

In [0]:
%run ../setup/00_setup

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import (
    StructType, StructField,
    StringType, TimestampType, DateType, LongType
)
from datetime import datetime, timezone

# ── KLUCZOWE USTAWIENIE: Spark zawsze operuje na UTC wewnętrznie.
# Ustawienie session.timeZone na UTC eliminuje niejednoznaczności
# przy wyświetlaniu i eksporcie danych między strefami czasowymi.
spark.conf.set("spark.sql.session.timeZone", "UTC")

# ── Ścieżki do demo archiwizacji (Part 2)
BONUS_BASE       = f"{DATASET_PATH}/bonus_demo"
SOURCE_PATH      = f"{BONUS_BASE}/source"
ARCHIVE_PATH     = f"{BONUS_BASE}/archive"
CHECKPOINT_PATH  = f"{BONUS_BASE}/checkpoint_archive"
SCHEMA_PATH      = f"{BONUS_BASE}/schema_archive"
TARGET_TABLE     = f"{CATALOG}.{BRONZE_SCHEMA}.bonus_orders_archive"

# Cleanup poprzedniego uruchomienia
dbutils.fs.rm(BONUS_BASE, True)
print(f"  Środowisko przygotowane: {BONUS_BASE}")
print(f"    spark.sql.session.timeZone = {spark.conf.get('spark.sql.session.timeZone')}")

### 1.1 Date vs Timestamp — podstawowe różnice

| Typ | Precyzja | Strefa czasowa | Użycie |
|-----|----------|----------------|--------|
| `DATE` | dzień | brak | daty kalendarzowe, partycjonowanie |
| `TIMESTAMP` / `TIMESTAMP_LTZ` | mikrosekundy | tak (UTC wewnętrznie) | zdarzenia z dokładnym czasem |
| `TIMESTAMP_NTZ` | mikrosekundy | brak | harmonogramy lokalne, etykiety czasowe |

> **Reguła:** Jeśli Twoje dane reprezentują **moment w czasie** (log, transakcja, zdarzenie) → `TIMESTAMP_LTZ` (UTC).  
> Jeśli reprezentują **datę lokalną** bez kontekstu strefy → `DATE` lub `TIMESTAMP_NTZ`.

In [0]:
# Dane wejściowe — zdarzenia z różnych systemów i stref czasowych
raw_data = [
    # (id, zdarzenie,      timestamp jako string,       strefa źródłowa)
    (1, "zamówienie PL", "2024-06-15 14:30:00",        "Europe/Warsaw"),
    (2, "zamówienie US", "2024-06-15 08:30:00",        "America/New_York"),
    (3, "zamówienie JP", "2024-06-15 21:30:00",        "Asia/Tokyo"),
    (4, "zamówienie UK", "2024-06-15 13:30:00",        "Europe/London"),
    (5, "zamówienie AU", "2024-06-15 23:30:00",        "Australia/Sydney"),
]

schema = StructType([
    StructField("id",            LongType(),   False),
    StructField("event",         StringType(), True),
    StructField("local_time_str", StringType(), True),
    StructField("source_tz",     StringType(), True),
])

df_raw = spark.createDataFrame(raw_data, schema)

# Parsujemy string do TIMESTAMP (Spark zakłada session.timeZone = UTC)
df_raw = df_raw.withColumn(
    "local_timestamp",
    F.to_timestamp("local_time_str", "yyyy-MM-dd HH:mm:ss")
)

display(df_raw)

### 1.2 Konwersja stref czasowych → UTC

**Problem:** Każdy system dostarczył timestamp w swojej strefie lokalnej. Musimy je wszystkie sprowadzić do UTC.

`from_utc_timestamp(ts, tz)` — konwertuje UTC → lokalna strefa  
`to_utc_timestamp(ts, tz)` — konwertuje lokalna strefa → UTC  
`convert_timezone(src_tz, tgt_tz, ts)` — nowszy wariant (SQL / Unity Catalog)

In [0]:
# ── to_utc_timestamp(col, tz): traktuje kolumnę jako czas w podanej strefie
# i przelicza go na UTC. To jest wzorzec dla danych wejściowych.
df_utc = (
    df_raw
    # Dla każdego rekordu znamy strefę źródłową — przeliczamy do UTC
    .withColumn(
        "event_utc",
        F.to_utc_timestamp("local_timestamp", F.col("source_tz"))
    )
    # Kolumna daty (tylko dzień) — przydatna do partycjonowania
    .withColumn("event_date", F.to_date("event_utc"))
)

display(
    df_utc.select(
        "id", "event", "source_tz",
        "local_time_str",
        "event_utc",
        "event_date"
    )
)

### 1.3 Wyświetlanie czasu lokalnego — `from_utc_timestamp`

Dane są przechowywane w UTC. Gdy użytkownik polskiego biura chce zobaczyć czas lokalny, konwertujemy **tylko przy wyświetlaniu** — nie zmieniamy wartości w tabeli.

In [0]:
# ── from_utc_timestamp(col, tz): przelicza UTC → podaną strefę lokalną
# Wzorzec: przechowuj UTC, konwertuj tylko dla prezentacji

TARGET_TIMEZONES = {
    "PL (Warsaw)":   "Europe/Warsaw",
    "US (New York)": "America/New_York",
    "JP (Tokyo)":    "Asia/Tokyo",
}

df_display = df_utc.select("id", "event", "event_utc")

for label, tz in TARGET_TIMEZONES.items():
    col_name = f"display_{label.split(' ')[0].lower()}"
    df_display = df_display.withColumn(
        col_name,
        F.date_format(
            F.from_utc_timestamp("event_utc", tz),
            "yyyy-MM-dd HH:mm:ss z"
        )
    )

display(df_display)

### 1.4 Pułapka: `session.timeZone` a parsowanie stringów

Jeśli nie ustawimy `spark.sql.session.timeZone = UTC`, Spark interpretuje stringe bez informacji o strefie jako **czas lokalny serwera klastra** (np. UTC-5 w AWS us-east).  
Godziny zostaną przesunięte przy odczycie/zapisie — bug trudny do wykrycia w produkcji.

In [0]:
TS_STRING = "2024-06-15 12:00:00"  # brak informacji o strefie

# ── Demonstracja wpływu session.timeZone na parsowanie
results = []
for tz_setting in ["UTC", "America/New_York", "Asia/Tokyo", "Europe/Warsaw"]:
    spark.conf.set("spark.sql.session.timeZone", tz_setting)
    val = spark.sql(f"SELECT CAST('{TS_STRING}' AS TIMESTAMP) AS ts").collect()[0]["ts"]
    results.append((tz_setting, str(val)))

# Przywracamy UTC (standard)
spark.conf.set("spark.sql.session.timeZone", "UTC")

df_tz_demo = spark.createDataFrame(results, ["session_timeZone", "parsed_timestamp_utc_repr"])
display(df_tz_demo)

print("\n  Przywrócono spark.sql.session.timeZone = UTC")

### 1.5 Best practices — tabela Delta z UTC

Rekomendowany schemat dla tabel produkcyjnych przyjmujących zdarzenia z różnych stref czasowych:

```
event_id          BIGINT
event_timestamp   TIMESTAMP    -- zawsze UTC (TIMESTAMP_LTZ)
event_date        DATE         -- partycja po UTC date
source_timezone   STRING       -- zachowujemy info o strefie źródłowej
ingested_at       TIMESTAMP    -- czas ingestii (current_timestamp() = UTC)
```

Nigdy nie mieszaj stref w jednej kolumnie. Zachowaj `source_timezone` jako metadane.

In [0]:
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

# Tworzymy tabelę Delta ze wzorcowym schematem UTC
spark.sql("DROP TABLE IF EXISTS bonus_events_utc")
spark.sql("""
    CREATE TABLE IF NOT EXISTS bonus_events_utc (
        event_id        BIGINT,
        event_timestamp TIMESTAMP,   -- zawsze UTC
        event_date      DATE,        -- partycja po UTC date
        source_timezone STRING,      -- strefa źródłowa jako metadane
        event_name      STRING,
        ingested_at     TIMESTAMP    -- czas ingestii (UTC)
    )
    USING DELTA
    PARTITIONED BY (event_date)
    TBLPROPERTIES (
        'delta.enableChangeDataFeed' = 'false',
        'comment' = 'Zdarzenia z wielu stref — timestamp zawsze UTC'
    )
""")

# Wstawiamy dane z wcześniej przygotowanego DataFrame
df_final = (
    df_utc
    .withColumnRenamed("id", "event_id")
    .withColumnRenamed("event", "event_name")
    .withColumnRenamed("event_utc", "event_timestamp")
    .withColumn("ingested_at", F.current_timestamp())
    .select("event_id", "event_timestamp", "event_date",
            "source_tz", "event_name", "ingested_at")
    .withColumnRenamed("source_tz", "source_timezone")
)

df_final.write.mode("append").saveAsTable("bonus_events_utc")

display(spark.sql("""
    SELECT
        event_id,
        event_name,
        source_timezone,
        event_timestamp                                          AS utc_time,
        from_utc_timestamp(event_timestamp, 'Europe/Warsaw')    AS warsaw_time,
        from_utc_timestamp(event_timestamp, 'America/New_York') AS new_york_time,
        from_utc_timestamp(event_timestamp, 'Asia/Tokyo')       AS tokyo_time
    FROM bonus_events_utc
    ORDER BY event_id
"""))

---

## Part 2: Auto Loader z Archiwizacją Plików

---

### Scenariusz

System zamówień co kilka minut zrzuca pliki JSON do folderu `source/`.  
Po przetworzeniu pliki muszą zostać **przeniesione do `archive/`**, zamiast usuwane — wymaganie compliance.

**Opcja dokumentacyjna:** [`cloudFiles.cleanSource = MOVE`](https://docs.databricks.com/aws/en/ingestion/cloud-object-storage/auto-loader/options#common-auto-loader-options)

| Opcja | Wartość | Opis |
|-------|---------|------|
| `cloudFiles.cleanSource` | `MOVE` | Przenieś pliki po przetworzeniu |
| `cloudFiles.cleanSource.moveDestination` | ścieżka | Gdzie przeносić pliki (musi być w tym samym bucket/container) |
| `cloudFiles.cleanSource.retentionDuration` | `0 hours` | Jak długo czekać przed przeniesieniem (domyślnie 30 dni; min. 0 dla MOVE) |

> **Uwaga:** Auto Loader musi mieć uprawnienia zapisu do `source/` i `archive/`. Folder `archive/` **nie może być podfolderem** `source/` — spowoduje ponowną ingestię.

### 2.1 Przygotowanie danych symulacyjnych

Tworzymy 3 mikro-batche plików JSON symulujące przybycie zamówień.

In [0]:
import json

# Wczytujemy istniejące pliki zamówień ze streamingu i dzielimy na 3 batche
SOURCE_STREAM_FILES = f"{DATASET_PATH}/orders/stream/*.json"
df_orders = spark.read.json(SOURCE_STREAM_FILES)
batches = df_orders.randomSplit([0.33, 0.33, 0.34], seed=42)

for i, batch_df in enumerate(batches, start=1):
    batch_df.coalesce(1).write.mode("overwrite").json(f"{SOURCE_PATH}/batch_{i:02d}")
    count = batch_df.count()
    print(f"  batch_{i:02d}: {count} rekordów → {SOURCE_PATH}/batch_{i:02d}")

# Sprawdzamy co jest w source
print("\nZawartość source/:")
for f in dbutils.fs.ls(SOURCE_PATH):
    print(f"  {f.path}")

### 2.2 Auto Loader z `cloudFiles.cleanSource = MOVE`

Kluczowe opcje:

- `cloudFiles.cleanSource` = `"MOVE"` — automatycznie przenosi pliki po przetworzeniu
- `cloudFiles.cleanSource.moveDestination` — folder docelowy (archiwum)
- `cloudFiles.cleanSource.retentionDuration` — czas przed przeniesieniem (min. `0 hours` dla MOVE)
- `cloudFiles.schemaLocation` — wymagane przy inferencji schematu
- `cloudFiles.inferColumnTypes` = `True` — inferuje typy (nie tylko String)

> **Ważne:** `retentionDuration` = `"0 hours"` oznacza przeniesienie natychmiast po potwierdzeniu commit przez strumień. Dostępne od **Databricks Runtime 16.4+**.

In [0]:
from pyspark.sql.streaming import StreamingQuery

# ── Tworzymy strumień Auto Loader z archiwizacją
df_stream = (
    spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format",                    "json")
    # ── Schemat i typy
    .option("cloudFiles.schemaLocation",            SCHEMA_PATH)
    .option("cloudFiles.inferColumnTypes",          "true")
    # ── Archiwizacja: przenieś przetworzone pliki do ARCHIVE_PATH
    .option("cloudFiles.cleanSource",               "MOVE")
    .option("cloudFiles.cleanSource.moveDestination", ARCHIVE_PATH)
    .option("cloudFiles.cleanSource.retentionDuration", "0 hours")  # DBR 16.4+
    # ── Kontrola przepustowości
    .option("cloudFiles.maxFilesPerTrigger",        "1")
    # ── Dodaje kolumnę _metadata z nazwą pliku, rozmiarem, czasem modyfikacji
    .load(SOURCE_PATH)
    # Dodajemy timestamp ingestii (UTC)
    .withColumn("ingested_at", F.current_timestamp())
    # Wyciągamy nazwę pliku źródłowego z metadanych
    .withColumn("source_file", F.col("_metadata.file_path"))
)

# ── Uruchamiamy w trybie Triggered (AvailableNow) — przetwarza wszystkie
# dostępne pliki i kończy się. Idealne do batch-like jobs w potoku.
query: StreamingQuery = (
    df_stream.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", CHECKPOINT_PATH)
    .option("mergeSchema", "true")
    .trigger(availableNow=True)
    .table(TARGET_TABLE)
)

query.awaitTermination()
print(f" Stream zakończony. Przetworzone rekordy zapisane do: {TARGET_TABLE}")

### 2.3 Weryfikacja — dane w tabeli i pliki w archiwum

In [0]:
# ── 1. Ile rekordów w tabeli docelowej?
count = spark.table(TARGET_TABLE).count()
print(f"Rekordów w tabeli '{TARGET_TABLE}': {count}")

# ── 2. Podgląd danych z kolumną source_file i ingested_at
display(
    spark.table(TARGET_TABLE)
    .limit(10)
)

# ── 3. Sprawdzamy czy source/ jest pusty (pliki przeniesione)
print("\nZawartość source/ po przetworzeniu:")
try:
    remaining = dbutils.fs.ls(SOURCE_PATH)
    for f in remaining:
        print(f"  {f.path}")
except Exception:
    print("  (folder pusty lub brak plików JSON)")

# ── 4. Sprawdzamy archiwum
print("\nZawartość archive/:")
for f in dbutils.fs.ls(ARCHIVE_PATH):
    print(f"  {f.path}")

### 2.4 Stan strumienia — `cloud_files_state`

Databricks udostępnia funkcję tabelaryczną `cloud_files_state(checkpoint_path)`,  
która pokazuje stan każdego pliku: `queued`, `processed`, `archived`.

In [0]:
# cloud_files_state — TVF zwraca metadane o każdym pliku przetworzonym przez Auto Loader
# Kolumny: path, timestamp, batchId, processedTimestamp, commit_time, size
display(
    spark.sql(f"""
        SELECT
            path,
            size,
            batchId,
            timestamp       AS file_modification_time,
            commit_time
        FROM cloud_files_state('{CHECKPOINT_PATH}')
        ORDER BY batchId, path
    """)
)

### 2.5 Alternatywa: Ręczna archiwizacja przez `foreachBatch`

Gdy potrzebujemy **zaawansowanej logiki archiwizacji** (np. foldery `archive/YYYY/MM/DD/`, różne reguły dla różnych plików, logowanie do tabeli audytowej), używamy `foreachBatch` zamiast `cloudFiles.cleanSource`.

> Używaj `cloudFiles.cleanSource = MOVE` dla prostych przypadków.  
> Używaj `foreachBatch` gdy potrzebujesz pełnej kontroli nad logiką przenoszenia.

---

## Cleanup

In [0]:
# Czyszczenie zasobów po demo
spark.sql("DROP TABLE IF EXISTS bonus_events_utc")
spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE}")
spark.sql(f"DROP TABLE IF EXISTS {TARGET_TABLE_V2}")
dbutils.fs.rm(BONUS_BASE, True)
print("✅  Cleanup zakończony")

---

## Podsumowanie — Cheatsheet

### Timestamps i UTC

| Zadanie | Funkcja Spark / SQL |
|---------|---------------------|
| Ustaw UTC jako standard klastra | `spark.conf.set("spark.sql.session.timeZone", "UTC")` |
| Parsuj string do timestamp | `to_timestamp(col, format)` |
| Lokalna → UTC | `to_utc_timestamp(col, "Europe/Warsaw")` |
| UTC → Lokalna (wyświetlanie) | `from_utc_timestamp(col, "Asia/Tokyo")` |
| Aktualny czas UTC | `current_timestamp()` |
| Tylko data z timestamp | `to_date(col)` |

### Auto Loader z archiwizacją

| Opcja | Wartość | Kiedy używać |
|-------|---------|--------------|
| `cloudFiles.cleanSource` | `"MOVE"` | Prosta archiwizacja (DBR 16.4+) |
| `cloudFiles.cleanSource.moveDestination` | ścieżka archiwum | Wymagane gdy `cleanSource=MOVE` |
| `cloudFiles.cleanSource.retentionDuration` | `"0 hours"` | Natychmiastowe przenoszenie |
| `foreachBatch` + `dbutils.fs.mv` | — | Zaawansowana logika, partycje datowe |

### Reguły złotego standardu

1. `spark.sql.session.timeZone = UTC` — **zawsze** w konfiguracji klastra/notatnika
2. Kolumna `event_timestamp` → **zawsze TIMESTAMP (UTC)**
3. Kolumna `source_timezone` → zachowaj jako metadane
4. Konwertuj do czasu lokalnego **tylko na potrzeby wyświetlania**
5. Partycjonuj po `event_date` (UTC) — nie po lokalnej dacie
6. `archive/` **nigdy** nie może być podfolderem `source/`